## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Hermanus,ZA,2022-01-29 21:09:31,-34.4187,19.2345,64.80,76,3,13.76,clear sky
1,1,Acapulco,MX,2022-01-29 21:09:32,16.8634,-99.8901,84.02,65,20,9.22,few clouds
2,2,Tasiilaq,GL,2022-01-29 21:09:32,65.6145,-37.6368,15.96,76,94,6.64,overcast clouds
3,3,Cape Town,ZA,2022-01-29 21:08:15,-33.9258,18.4232,68.11,72,0,4.00,clear sky
4,4,Codrington,AU,2022-01-29 21:09:32,-38.2667,141.9667,63.81,82,80,6.33,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Acapulco,MX,2022-01-29 21:09:32,16.8634,-99.8901,84.02,65,20,9.22,few clouds
5,5,Arraial Do Cabo,BR,2022-01-29 21:09:33,-22.9661,-42.0278,75.06,83,75,9.22,light rain
8,8,Rikitea,PF,2022-01-29 21:09:33,-23.1203,-134.9692,77.32,73,100,10.71,overcast clouds
11,11,Pitimbu,BR,2022-01-29 21:09:34,-7.4706,-34.8086,79.39,78,66,7.81,broken clouds
12,12,San Pedro,PH,2022-01-29 21:09:35,14.3500,121.0167,79.36,90,3,3.04,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                166
City                   166
Country                166
Date                   166
Lat                    166
Lng                    166
Max Temp               166
Humidity               166
Cloudiness             166
Wind Speed             166
Weather Description    166
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Acapulco,MX,84.02,few clouds,16.8634,-99.8901,
5,Arraial Do Cabo,BR,75.06,light rain,-22.9661,-42.0278,
8,Rikitea,PF,77.32,overcast clouds,-23.1203,-134.9692,
11,Pitimbu,BR,79.39,broken clouds,-7.4706,-34.8086,
12,San Pedro,PH,79.36,clear sky,14.3500,121.0167,
16,Villa Carlos Paz,AR,81.25,clear sky,-31.4241,-64.4978,
23,Cap Malheureux,MU,79.47,overcast clouds,-19.9842,57.6142,
35,Hithadhoo,MV,80.87,overcast clouds,-0.6000,73.0833,
37,Auki,SB,80.10,overcast clouds,-8.7676,160.7034,
42,Bengkulu,ID,75.88,heavy intensity rain,-3.8004,102.2655,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Acapulco,MX,84.02,few clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
5,Arraial Do Cabo,BR,75.06,light rain,-22.9661,-42.0278,Pousada Porto Praia
8,Rikitea,PF,77.32,overcast clouds,-23.1203,-134.9692,People ThankYou
11,Pitimbu,BR,79.39,broken clouds,-7.4706,-34.8086,Reserva do Abiaí
12,San Pedro,PH,79.36,clear sky,14.3500,121.0167,Villa Veronica Resort Private Pool
...,...,...,...,...,...,...,...
681,Upington,ZA,84.58,overcast clouds,-28.4478,21.2561,River Place Manor
689,Bairiki,KI,84.20,light rain,1.3292,172.9752,Tarawa Boutique Hotel
690,Kulhudhuffushi,MV,81.82,broken clouds,6.6221,73.0700,Haajy Guest House 1
693,Ayer Tawar,MY,77.29,few clouds,4.2954,100.7548,Ayer Tawar CK Homestay


In [25]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
path=r'C:\Users\carla\OneDrive\Documents\GitHub\world_weather_analysis\vacation_search'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(os.path.join(path,output_data_file),index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
weather_descript = clean_hotel_df['Weather Description']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))